In [18]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os
import random
import cv2
import matplotlib
from sklearn.model_selection import train_test_split
from skimage.filters import threshold_otsu
import torchmetrics
import pickle
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torchvision.models as models
import torchvision
from torchvision import transforms
import torch
import torch.nn as nn
from torch.optim import Adam
from torch import LongTensor as LongTensor
from torch import FloatTensor as FloatTensor
import cv2

In [19]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## Load data

In [20]:
train_set_path = "./Data/raw_data/train.csv"
train_tiles_encoding_folder = "./Data/processed_data/train_tiles_encoding/"

test_set_path = "./Data/raw_data/test.csv"
test_tiles_encoding_folder = "./Data/processed_data/test_tiles_encoding/"

In [21]:
train_set = pd.read_csv(train_set_path)
test_set = pd.read_csv(test_set_path)

In [22]:
## keep only if enough tiles retrieve
to_remove = []
for idx in range(train_set.shape[0]):
  value = train_set.iloc[idx]
  image_id = value.image_id
  # compute file
  image_embeddings_path = train_tiles_encoding_folder+image_id+'.pkl'

  # open embeddings  dict
  embeddings_dict = pickle.load(open(image_embeddings_path, 'rb'))
  if len(embeddings_dict) < 180:
    to_remove.append(image_id)

In [23]:
train_set = train_set.query('image_id not in @to_remove')

## Prepare Data

In [24]:
validation_ratio = 0.1

X = train_set.drop(columns = ['isup_grade'])
y = train_set[['isup_grade']]
train_samples, validation_samples, train_labels, validation_labels = train_test_split(X,y, test_size=validation_ratio, random_state=0, shuffle=True, stratify = y)

train_set = pd.concat([train_samples, train_labels], axis = 1)
validation_set = pd.concat([validation_samples, validation_labels], axis = 1)

## Define epoch training

In [25]:
# 1 epoch training

def train_one_epoch(model, trainloader, validationloader, optimizer, device): 
    losses = []

    val_auroc = torchmetrics.AUROC(num_classes = 6)
    val_f1 = torchmetrics.F1Score(num_classes = 6)
    train_f1 = torchmetrics.F1Score(num_classes = 6)
    best_validation_f1 = -np.inf

    ### traning 
    model.train()
    for (features, target) in tqdm(trainloader):
        features, target = features.to(device), target.to(device)

        optimizer.zero_grad()
        predictions = model(features)
        predicted_classes = torch.argmax(predictions, dim=1)
      
        criterion = nn.CrossEntropyLoss()
        loss = criterion(predictions, target)
        losses.append(float(loss))
        loss.backward()
        optimizer.step()
        f1_train = train_f1(predicted_classes.cpu(), target.cpu())

    ### model evaluation 
    model.eval()
    with torch.no_grad():
      for (features, target) in (validationloader):
          features, target = features.to(device), target.to(device)

          predictions = model(features)
          predicted_classes = torch.argmax(predictions, dim=1)
          
          validation_auroc = val_auroc(predictions.cpu().detach(), target.cpu().detach())
          f1_val = val_f1(predicted_classes.cpu().detach(), target.cpu().detach())


    val_f1_final = val_f1.compute()    
    train_f1_final = train_f1.compute()  
    val_auroc_final = val_auroc.compute()
    print('average train loss: ', np.mean(losses))
    print('validation f1: ', val_f1_final)
    print('train f1: ', train_f1_final)
    print('validation auroc: ', val_auroc_final)

    return val_auroc_final

In [26]:
def train_model(model, trainloader, validationloader, optimizer, device, n_epoch, checkpoint_path, early_stopping):
    model.to(device)

    
    best_roc = -np.inf
    previous_roc = -np.inf
    counter = 0
    for epoch in range(0, n_epoch):
        print(f"epoch {epoch+1}/{n_epoch}")
        roc = train_one_epoch(model , trainloader, validationloader, optimizer, device)
        if roc >= best_roc:
            best_roc = roc
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'roc': roc,
            }, checkpoint_path)

            print('New best model saved !')

        if roc <= previous_roc:
            print('no iprovement')
            counter += 1
        
        else:
            counter = 0

        previous_roc = roc

        if counter == early_stopping:
            print('early stopping !')
            break 
    print("load best model")

    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return(model)       

## Define data sets


In [27]:
class EmbeddingDatasetNoPosition(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df, dir, bag_size = 180):
        """
        Args:
            df (dataframe): Path to the csv file with annotations.
            dir (string): Directory with tiles images embeddings
        """
        self.df = df
        self.dir = dir
        self.bag_size = bag_size

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        value = self.df.iloc[idx]
        image_id = value.image_id
        # compute file
        image_embeddings_path = self.dir+image_id+'.pkl'

        # open embeddings  dict
        embeddings_dict = pickle.load(open(image_embeddings_path, 'rb'))

        bag = None
        embedding_list = np.array(list(embeddings_dict.values()))
        np.random.shuffle(embedding_list)
        bag = np.vstack(embedding_list[:self.bag_size])

        bag = torch.Tensor(bag)
        # compute label
        label = torch.tensor(value.isup_grade)

        return bag, label

## Define Models

In [28]:
class ModelVanilla(nn.Module):
    def __init__(self, number_of_embeddings = 180, output_shape = 6, embedding_shape = 1280):
        super(ModelVanilla, self).__init__()
        
        # to encode each bag
        self.aggregator = torch.nn.AvgPool1d(number_of_embeddings)


        self.classifier = nn.Sequential(
                        nn.Linear(embedding_shape, int(embedding_shape/2)),
                        nn.ReLU(),
                        torch.nn.BatchNorm1d(int(embedding_shape/2)),
                        nn.Dropout(0.25),
                        nn.Linear(int(embedding_shape/2), int(embedding_shape/4)),
                        nn.ReLU(),
                        nn.Dropout(0.25),
                        torch.nn.BatchNorm1d(int(embedding_shape/4)),
                        nn.Linear(int(embedding_shape/4), output_shape)
                        )
        
        
        self.softmax = nn.Softmax(dim = 1)


        
    def forward(self, bag):
      bag = torch.transpose(bag, 1, 2)
      wsi_descriptor = self.aggregator(bag).squeeze(-1)
      out = self.classifier(wsi_descriptor)
      out = self.softmax(out)
      return out

In [29]:
class AttentionModel(nn.Module):
    def __init__(self, number_of_embeddings = 180, output_shape = 6, embedding_shape = 1280):
        super(AttentionModel, self).__init__()
        
        # to compute attenation map for aggregation
        self.attention = nn.Sequential(
            nn.Linear(embedding_shape, int(embedding_shape/4)),
            nn.Tanh(),
            nn.Linear(int(embedding_shape/4), 1)
            )


        self.classifier = nn.Sequential(
                        nn.Linear(embedding_shape, int(embedding_shape/2)),
                        nn.ReLU(),
                        torch.nn.BatchNorm1d(int(embedding_shape/2)),
                        nn.Dropout(0.25),
                        nn.Linear(int(embedding_shape/2), int(embedding_shape/4)),
                        nn.ReLU(),
                        torch.nn.BatchNorm1d(int(embedding_shape/4)),
                        nn.Dropout(0.25),
                        nn.Linear(int(embedding_shape/4), output_shape)
        )
        
        
        self.softmax = nn.Softmax(dim = 1)


        
    def forward(self, bag):
      # compute attention map
      attention_map = self.attention(bag)
      attention_map = torch.transpose(attention_map, 2, 1) # KxN
      attention_map = self.softmax(attention_map)
      # apply attention map
      M = torch.matmul(attention_map, bag).squeeze(1)
      # classification
      out = self.classifier(M)
      out = self.softmax(out)
      return out

In [30]:
class ModelChowder(nn.Module):
    def __init__(self, number_of_embeddings = 180, output_shape = 6, embedding_shape = 1280, R = 5):
        super(ModelChowder, self).__init__()
        
        # to encode each bag
        self.R = R
        self.conv = torch.nn.Conv1d(number_of_embeddings,number_of_embeddings,kernel_size =embedding_shape)
        
        self.classifier = nn.Sequential(
                        nn.Linear(2*self.R, 200),
                        nn.Sigmoid(),
                        torch.nn.BatchNorm1d(200),
                        nn.Dropout(0.25),
                        nn.Linear(200, 100),
                        nn.Sigmoid(),
                        torch.nn.BatchNorm1d(100),
                        nn.Dropout(0.25),
                        nn.Linear(100, output_shape)
                        )
        
        
        self.softmax = nn.Softmax(dim = 1)


        
    def forward(self, bag):
      post_conv = self.conv(bag).squeeze(-1)
      post_conv = torch.sort(post_conv)[0]

      max_values = post_conv[:,-self.R:]
      min_values = post_conv[:,:self.R]

      wsi_descriptor = torch.cat((max_values,min_values), 1)



      out = self.classifier(wsi_descriptor)
      out = self.softmax(out)
      return out

## Train models

In [31]:
batch_size = 16
number_of_tiles_per_bag = 20
train_set = EmbeddingDatasetNoPosition(train_set, train_tiles_encoding_folder, bag_size = number_of_tiles_per_bag)
validation_set = EmbeddingDatasetNoPosition(validation_set, train_tiles_encoding_folder, bag_size = number_of_tiles_per_bag)


trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last = True)
validationloader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=True, drop_last = True)


In [32]:
model = ModelChowder(number_of_embeddings = number_of_tiles_per_bag)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)
checkpoint_path_chowder = "./Data/checkpoints/model_chowder"
best_model = train_model(model, trainloader, validationloader, optimizer, device, 100, checkpoint_path_chowder, 5)

c:\Users\33631\Documents\Etudes\centrale 3A\Deep Learning for medical Imaging\prostate_cancer_detection\venv\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


epoch 1/100


100%|██████████| 19/19 [00:01<00:00, 12.49it/s]
c:\Users\33631\Documents\Etudes\centrale 3A\Deep Learning for medical Imaging\prostate_cancer_detection\venv\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


average train loss:  1.7963831424713135
validation f1:  tensor(0.1562)
train f1:  tensor(0.1842)
validation auroc:  tensor(0.6403)
New best model saved !
epoch 2/100


100%|██████████| 19/19 [00:01<00:00, 14.37it/s]


...

average train loss:  1.6429106310794228
validation f1:  tensor(0.3438)
train f1:  tensor(0.4145)
validation auroc:  tensor(0.7424)
no iprovement
epoch 74/100


100%|██████████| 19/19 [00:02<00:00,  8.01it/s]


average train loss:  1.6196934612173783
validation f1:  tensor(0.3125)
train f1:  tensor(0.4638)
validation auroc:  tensor(0.7637)
epoch 75/100


 26%|██▋       | 5/19 [00:00<00:02,  6.94it/s]


KeyboardInterrupt: 

In [33]:
model = AttentionModel(number_of_embeddings = number_of_tiles_per_bag)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)
checkpoint_path_attention = "./Data/checkpoints/model_attention"
best_model = train_model(model, trainloader, validationloader, optimizer, device, 100, checkpoint_path_attention, 5)

epoch 1/100


100%|██████████| 19/19 [00:01<00:00, 16.09it/s]


average train loss:  1.7783041565041793
validation f1:  tensor(0.2500)
train f1:  tensor(0.2007)
validation auroc:  tensor(0.5649)
New best model saved !
epoch 2/100


100%|██████████| 19/19 [00:01<00:00, 18.08it/s]


average train loss:  1.7559023593601428
validation f1:  tensor(0.2500)
train f1:  tensor(0.2862)
validation auroc:  tensor(0.5698)
New best model saved !
epoch 3/100


...

average train loss:  1.518570994075976
validation f1:  tensor(0.5000)
train f1:  tensor(0.5855)
validation auroc:  tensor(0.8015)
epoch 99/100


100%|██████████| 19/19 [00:01<00:00, 10.42it/s]


average train loss:  1.526691474412617
validation f1:  tensor(0.3750)
train f1:  tensor(0.5461)
validation auroc:  tensor(0.7606)
no iprovement
epoch 100/100


100%|██████████| 19/19 [00:01<00:00, 10.64it/s]


average train loss:  1.5132911895450794
validation f1:  tensor(0.4375)
train f1:  tensor(0.5559)
validation auroc:  tensor(0.7475)
no iprovement
load best model


In [ ]:
model = ModelVanilla(number_of_embeddings = number_of_tiles_per_bag)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)
checkpoint_path_vanilla = "./Data/checkpoints/model_vanilla"
best_model = train_model(model, trainloader, validationloader, optimizer, device, 100, checkpoint_path_vanilla, 5)

epoch 1/100


100%|██████████| 19/19 [00:01<00:00, 10.83it/s]


average train loss:  1.782977675136767
validation f1:  tensor(0.2500)
train f1:  tensor(0.2072)
validation auroc:  tensor(0.5777)
New best model saved !
epoch 2/100


100%|██████████| 19/19 [00:01<00:00, 10.11it/s]


average train loss:  1.7662819310238487
validation f1:  tensor(0.2812)
train f1:  tensor(0.2467)
validation auroc:  tensor(0.6170)
New best model saved !
epoch 3/100


...

average train loss:  1.6537839550721019
validation f1:  tensor(0.2812)
train f1:  tensor(0.4474)
validation auroc:  tensor(0.7261)
no iprovement
epoch 14/100


100%|██████████| 19/19 [00:02<00:00,  7.56it/s]


average train loss:  1.6521120447861521
validation f1:  tensor(0.3125)
train f1:  tensor(0.4408)
validation auroc:  tensor(0.7379)
New best model saved !
epoch 15/100


100%|██████████| 19/19 [00:02<00:00,  7.06it/s]


average train loss:  1.6601589165235822
validation f1:  tensor(0.2500)
train f1:  tensor(0.4178)
validation auroc:  tensor(0.7330)
no iprovement
early stopping !
load best model


# Prediction

In [ ]:
bag_size = 170
final_df = pd.DataFrame(columns = ['Id','Predicted'])
for i in range(test_set.shape[0]):
  value = test_set.iloc[i]
  id = value.image_id


  # compute file
  image_embeddings_path = test_tiles_encoding_folder+id+'.pkl'

  # open embeddings  dict
  embeddings_dict = pickle.load(open(image_embeddings_path, 'rb'))

  bag = None
  embedding_list = np.array(list(embeddings_dict.values()))
  np.random.shuffle(embedding_list)
  bag = np.vstack(embedding_list[:bag_size])

  bag = torch.Tensor(bag).unsqueeze(0).to(device)

  # get prediction
  model.eval()
  pred = model(bag).detach().cpu().numpy()

  final_pred = int(np.argmax(pred))
  current_line = pd.DataFrame([[id, final_pred]], columns = ['Id','Predicted'])
  final_df = pd.concat([final_df, current_line], axis = 0)


final_df.to_csv("/content/drive/MyDrive/centrale_3A/Deep Learning for medical imaging/final_preds.csv", index = False)
